### **<u>Microsoft Fabric Spark Catalog</u>**
The Microsoft Fabric Spark Catalog is a component of Microsoft Fabric, an Azure-integrated platform based on Apache Spark. 
It enables the execution and management of data engineering and data science tasks. Here are some key features:

- **Data Management**: The Spark Catalog allows you to manage various types of tables, including managed, unmanaged, and Delta tables.
- **Integration with Apache Spark**: It leverages Apache Spark's powerful distributed computing capabilities for large-scale data processing and analytics.
- **Support for Multiple Formats**: You can create tables in various formats such as Parquet, CSV, ORC, Avro, and Delta Lake.
- <u>**SQL Interactions**: The tables created in the Spark Catalog can be queried using SQL, making it easier to interact with and analyze your data</u>

##### <u><span style="color:blue">**Microsoft Fabric Spark Catalog: Managed vs Unmanaged Tables**</span></u>

###### <span style="color:green">**Managed Tables:**</span>
- **Data and Metadata Management**: Spark manages both data and metadata.
- **Storage**: Data resides in a storage directory managed by Spark.
- **Deletion**: Dropping a managed table deletes both data and metadata.

###### <span style="color:red">**Unmanaged (External) Tables:**</span>
- **Metadata Management**: Spark manages only the metadata.
- **Storage**: Data resides in an external location specified by the user.
- **Deletion**: Dropping an unmanaged table deletes only the metadata; the data remains intact.

##### **<span style="color:brown">Microsoft Fabric Spark Catalog: Non-Persisted vs Persisted</span>**
- Microsoft Fabric Spark Catalog: Non-Persisted vs Persisted Tables

<u>**Non-Persisted Tables**</u>
- **Purpose**: Used for temporary data storage within a session.
- **Usage**: Ideal for intermediate computations and temporary data processing.
- **Example**: Creating a temporary table to hold intermediate results during a complex data transformation process.
- **Read-Only**: Cannot directly update or insert rows.
  ```python
  df.createOrReplaceTempView("temp_view")
  
  df.createOrReplaceGlobalTempView("global_temp_view")

<u>**Persisted Tables**</u>
- **Purpose**: Used for permanent data storage.
- **Usage**: Suitable for data that needs to be stored and accessed over multiple sessions.
- **Example**: Creating a table to store customer transaction records.
  ```python
  df.write.format("delta").saveAsTable("customer_transactions")

##### **<u><span style="color:blue">Microsoft Fabric Spark Catalog: Appendable vs Transactional Tables</span></u>**

##### **<span style="color:green">Appendable Tables</span>**
- **Purpose**: Designed for scenarios where data is continuously added.
- **Usage**: Ideal for logging or streaming data.
- **Example**: A table that stores log entries from an application, where new logs are appended as they are generated.

##### **<span style="color:red">Transactional Tables</span>**
- **Purpose**: Supports transactions to ensure data consistency.
- **Usage**: Suitable for operations requiring atomicity, consistency, isolation, and durability (ACID).
- **Example**: A table that handles purchase orders, where multiple related changes must be committed together to maintain data integrity.

##### **<u><span style="color:blue">Microsoft Fabric Spark Catalog - Appendable: Example</span></u>**

In [3]:
%%sql
DROP TABLE IF EXISTS customer_csv;

CREATE TABLE customer_csv (
    id INT,
    name VARCHAR(50),
    age INT
)
USING CSV
OPTIONS (
    header 'true',
    inferSchema 'true'
)
TBLPROPERTIES (
    'creator' = 'Jean Joseph',
    'purpose' = 'Store customer information for data tobbogan live demo'
);

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
%%sql
-- Verify table creation
--SHOW TABLES;
DESCRIBE EXTENDED customer_csv;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 7, Finished, Available, Finished)

<Spark SQL result set with 17 rows and 3 fields>

In [5]:
%%sql
SELECT * FROM customer_csv;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 3 fields>

In [6]:
%%sql
INSERT INTO customer_csv (id, name, age)
VALUES (1, 'John Doe', 30);

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
%%sql
SELECT * FROM customer_csv;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 3 fields>

In [8]:
%%sql
--cant update and delete
UPDATE customer_csv 
SET name = 'Jean Joseph'
WHERE id=1;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 11, Finished, Available, Finished)

Error: UPDATE TABLE is not supported temporarily.

#### **Unmanaged tables are not supported in the Microsoft Fabric Spark Catalog for appendable tables**


In [9]:
%%sql
DROP TABLE IF EXISTS customer_csv2;

CREATE TABLE customer_csv2 (
    id INT,
    name VARCHAR(50),
    age INT
)
USING CSV
OPTIONS (
    path 'abfss://community@onelake.dfs.fabric.microsoft.com/LH01.Lakehouse/Files/sparksql/csv/customers.csv',
    header 'true',
    inferSchema 'true'
)
TBLPROPERTIES (
    'creator' = 'Jean Joseph',
    'purpose' = 'Store customer information for data tobbogan live demo'
);

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 13, Finished, , Finished)

<Spark SQL result set with 0 rows and 0 fields>

Error: assertion failed: Only managed tables are supported

##### **The multiline option in Spark**
- The multiline option in Spark is used to correctly read JSON files that span multiple lines. By default, Spark expects each JSON object to be on a single line.
- When you set spark.read.option("multiline", "true"), Spark treats the entire content of the file as a single JSON object, allowing it to correctly parse JSON files that are formatted across multiple lines. This is particularly useful for handling complex JSON structures that cannot be represented on a single line.

**Array Functions:**

- **EXPLODE()**: Creates a new row for each element in an array.
- **SIZE()**: Returns the length of an array.
- **ARRAY_CONTAINS()**: Checks if an array contains a specific element.
- **ARRAY_DISTINCT()**: Removes duplicate elements from an array.

**JSON Functions:**

- **GET_JSON_OBJECT()**: Extracts a JSON object from a JSON string.
- **JSON_TUPLE()**: Extracts multiple values from a JSON string.
- **FROM_JSON()**: Parses a JSON string and returns a struct.
- **TO_JSON()**: Converts a struct to a JSON string.
You can copy and paste this into your notebook to have a nicely formatted reference for these functions. If you need any more help or additional information, feel free to ask!


In [10]:
# Read the JSON file into a DataFrame
df = spark.read.option("multiline", "true").json("abfss://community@onelake.dfs.fabric.microsoft.com/LH01.Lakehouse/Files/sparksql/json/sales_00.json")

# Save the DataFrame as a Spark SQL table with overwrite mode
df.write.mode("overwrite").format("json").saveAsTable("sales_json")

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 15, Finished, Available, Finished)

In [11]:
%%sql
DESCRIBE EXTENDED sales_json;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 16, Finished, Available, Finished)

<Spark SQL result set with 18 rows and 3 fields>

In [12]:
%%sql
select * from sales_json;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 17, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 5 fields>

In [13]:
%%sql
SELECT name, 
       age, 
       email,
       courses
FROM sales_json;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 18, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 4 fields>

In [14]:
%%sql
SELECT split(name,' ')[0] AS FirstName, 
       split(name,' ')[1] AS LastName,
       age, 
       email,
       courses.courseName,
       courses.courseCode,   
       courses.grade            
FROM sales_json;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 19, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 7 fields>

In [15]:
%%sql
SELECT split(name,' ')[0] AS FirstName, 
       split(name,' ')[1] AS LastName,
       age, 
       email,
       explode(courses.courseName) AS courseName,
       courses.courseCode,
       courses.grade       
FROM sales_json;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 20, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 7 fields>

In [16]:
%%sql
--will fail
SELECT name, 
       age, 
       email,
       explode(courses.courseName),
       explode(courses.courseCode),   
       explode(courses.grade)            
FROM sales_json;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 21, Finished, Available, Finished)

Error: [UNSUPPORTED_GENERATOR.MULTI_GENERATOR] The generator is not supported: only one generator allowed per SELECT clause but found 3: "explode(courses.courseName)", "explode(courses.courseCode)", "explode(courses.grade)".

In [17]:
%%sql
SELECT split(name,' ')[0] AS FirstName, 
       split(name,' ')[1] AS LastName,
       age, 
       email,
       courseName,
       courseCode,
       grade             
FROM sales_json
LATERAL VIEW explode(courses.courseName) AS courseName
LATERAL VIEW explode(courses.courseCode) AS courseCode
LATERAL VIEW explode(courses.grade) AS grade
LIMIT 10;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 22, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 7 fields>

In [18]:
%%sql
--List students who have taken more than one course:
SELECT name, COUNT(course.courseName) AS course_count
FROM sales_json
LATERAL VIEW explode(courses) AS course
GROUP BY name
HAVING course_count > 1

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 23, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 2 fields>

In [ ]:
%%sql
--Get the names and emails of students who scored an 'A' in any course:
SELECT DISTINCT name, email
FROM sales_json
LATERAL VIEW explode(courses) AS course
WHERE course.grade = 'A'

In [19]:
%%sql
--Calculate the average grade for each course:
SELECT course.courseName, AVG(CASE 
    WHEN course.grade = 'A+' THEN 4.0
    WHEN course.grade = 'A' THEN 4.0
    WHEN course.grade = 'A-' THEN 3.7
    WHEN course.grade = 'B+' THEN 3.3
    WHEN course.grade = 'B' THEN 3.0
    ELSE 0.0
END) AS average_grade
FROM sales_json
LATERAL VIEW explode(courses) AS course
GROUP BY course.courseName

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 24, Finished, Available, Finished)

<Spark SQL result set with 8 rows and 2 fields>

In [ ]:
%%sql
--Find the student with the highest average grade:
SELECT name, AVG(CASE 
    WHEN course.grade = 'A+' THEN 4.0
    WHEN course.grade = 'A' THEN 4.0
    WHEN course.grade = 'A-' THEN 3.7
    WHEN course.grade = 'B+' THEN 3.3
    WHEN course.grade = 'B' THEN 3.0
    ELSE 0.0
END) AS average_grade
FROM sales_json
LATERAL VIEW explode(courses) AS course
GROUP BY name
ORDER BY average_grade DESC
LIMIT 1

In [ ]:
%%sql
--Count the number of students enrolled in each course:
SELECT course.courseName, COUNT(*) AS student_count
FROM sales_json
LATERAL VIEW explode(courses) AS course
GROUP BY course.courseName

In [ ]:
%%sql
SELECT course.courseName
FROM sales_json
LATERAL VIEW explode(courses) AS course
--WHERE course.courseName LIKE 'Machine%'
--***CASE SENTITIVE
--WHERE course.courseName LIKE 'machine%'
WHERE LOWER(course.courseName) LIKE 'machine%'

In [ ]:
%%sql
SELECT email,
       INSTR(email, '@')
FROM sales_json
WHERE INSTR(email, '@') > 0

In [ ]:
%%sql
--extract email domain
SELECT 
  SUBSTRING(email, INSTR(email, '@') + 1) AS domain
FROM sales_json

In [ ]:
%%sql
----extract name from email
SELECT 
  SUBSTRING(email, 1, INSTR(email, '@') - 1) AS username
FROM sales_json

**Loop through each row in the result**
- Apache Spark SQL itself doesn't directly support looping through rows like traditional SQL.

Using `collect()`:
```python
rows = df.collect()
for row in rows:
    print(row)



Using `toLocalIterator()`: 
```python
for row in df.toLocalIterator():
    print(row)

Using `foreach()` with RDDs:
```python
df.rdd.foreach(lambda row: print(row))

Using `map()` Transformation:
```python
transformed_rdd = df.rdd.map(lambda row: (row['Name'], row['Id'] * 10))
transformed_df = spark.createDataFrame(transformed_rdd)
transformed_df.show()

In [20]:
%%pyspark
# Define the SQL query
sql_query = """
SELECT 
  ROW_NUMBER() OVER (ORDER BY email) AS Id,
  SUBSTRING(email, 1, INSTR(email, '@') - 1) AS username
FROM sales_json
"""

# Execute the query
result = spark.sql(sql_query)

# Display the result
result.show()

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 25, Finished, Available, Finished)

+---+------------+
| Id|    username|
+---+------------+
|  1|alicejohnson|
|  2|    bobbrown|
|  3|charliedavis|
|  4|   janesmith|
|  5|     johndoe|
+---+------------+



In [21]:
# Collect the result as a list of Row objects
rows = result.collect()

# Loop through each row and print the Id and username
for row in rows:
    print(f"Id: {row.Id}, Username: {row.username}")
    # Define the SQL query using an f-string
    sql_query = f"SELECT name, age, email FROM sales_json WHERE email LIKE '%{row.username}%'"
    
    result = spark.sql(sql_query)
    # Display the result
    result.show()

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 26, Finished, Available, Finished)

Id: 1, Username: alicejohnson
+-------------+---+--------------------+
|         name|age|               email|
+-------------+---+--------------------+
|Alice Johnson| 25|alicejohnson@exam...|
+-------------+---+--------------------+

Id: 2, Username: bobbrown
+---------+---+--------------------+
|     name|age|               email|
+---------+---+--------------------+
|Bob Brown| 28|bobbrown@example.com|
+---------+---+--------------------+

Id: 3, Username: charliedavis
+-------------+---+--------------------+
|         name|age|               email|
+-------------+---+--------------------+
|Charlie Davis| 24|charliedavis@exam...|
+-------------+---+--------------------+

Id: 4, Username: janesmith
+----------+---+--------------------+
|      name|age|               email|
+----------+---+--------------------+
|Jane Smith| 22|janesmith@example...|
+----------+---+--------------------+

Id: 5, Username: johndoe
+--------+---+-------------------+
|    name|age|              email|
+--

##### **<span style="color:red">Transactional Tables</span>**
<u>**Persisted Transactional Tables**</u>
- **Purpose**: Used for permanent data storage.
- **Usage**: Suitable for data that needs to be stored and accessed over multiple sessions.
- **Example**: Creating a table to store customer transaction records.
  ```python
  df.write.format("delta").saveAsTable("customer_transactions")

###### **<span style="color:brown">Transactional Tables Creation</span>**

In [22]:
%%sql
DROP TABLE IF EXISTS customer_delta;

CREATE TABLE customer_delta (
    id INT,
    name VARCHAR(50),
    age INT
)
USING DELTA
OPTIONS (
    header 'true',
    inferSchema 'true'
)
TBLPROPERTIES (
    'creator' = 'Jean Joseph',
    'purpose' = 'Store customer information for data tobbogan live demo'
);

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 28, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [23]:
%%sql
INSERT INTO customer_delta (id, name, age)
VALUES (1, 'John Doe', 30);

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 29, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [24]:
%%sql
SELECT * FROM customer_delta;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 30, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 3 fields>

In [25]:
%%sql
--can perform update and delete
UPDATE customer_delta 
SET name = 'Jean Joseph'
WHERE id=1;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 31, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

Turn csv into a Transaction Table (Delta)

In [26]:
# Read the CSV file into a DataFrame
csv_file_path = "abfss://community@onelake.dfs.fabric.microsoft.com/LH01.Lakehouse/Files/sparksql/csv/SalesOrderHeader.csv"

# Read the CSV file into a DataFrame with schema inference
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(csv_file_path)

# Save the DataFrame as a Delta table with overwrite mode
df.write.format("delta").mode("overwrite").saveAsTable("SalesOrderHeader")

#df.printSchema()

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 32, Finished, Available, Finished)

In [27]:
%%sql
SELECT YEAR(OrderDate) AS OrderDate,
       SUM(TotalDue) AS TotalSalesByYear,
       -- AVG(IFNULL(SalesOrderId,0)) AS AvgSalesByYear,
       COUNT(IFNULL(SalesOrderId,0)) AS SalesOrderByYear,
       RANK() OVER(ORDER BY SUM(TotalDue) DESC) AS RankSalesOrderByYear,
       PERCENT_RANK() OVER(ORDER BY SUM(TotalDue)) AS PercentRankSalesOrderByYear
FROM SalesOrderHeader
GROUP BY YEAR(OrderDate);

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 33, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 5 fields>

### **Handling Explicit Transactions in Apache Spark SQL**

Spark SQL doesn't natively support explicit transaction control commands like `BEGIN`, `COMMIT`, and `ROLLBACK` as traditional SQL databases do. However, you can achieve transactional-like behavior using Delta Lake, which provides ACID transactions.

##### **Start a Transaction**
- Delta Lake automatically handles transactions for you. When you perform operations like `INSERT`, `UPDATE`, or `DELETE`, Delta Lake ensures that these operations are atomic, consistent, isolated, and durable (ACID).

##### **Rollback a Transaction**
- Delta Lake supports time travel, which allows you to query previous versions of your data. You can use this feature to rollback to a previous state.

In [28]:
%%sql
--will fail
BEGIN TRANSACTION;

-- Delete the record with id=1
DELETE FROM customer_delta 
WHERE id = 1;

-- Rollback the transaction
ROLLBACK TRANSACTION;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 34, Finished, , Finished)

Error: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'BEGIN'.(line 2, pos 0)

== SQL ==
--will fail
BEGIN TRANSACTION
^^^


In [29]:
%%sql
-- Get the history of the table
DESCRIBE HISTORY customer_delta;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 35, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 15 fields>

In [30]:
%%sql
SELECT * FROM customer_delta;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 36, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 3 fields>

In [31]:
%%sql
-- Query the table as it was at a specific version
SELECT * FROM customer_delta VERSION AS OF 1;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 37, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 3 fields>

In [32]:
%%sql
-- Rollback to a specific version
RESTORE TABLE customer_delta TO VERSION AS OF 1;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 38, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 6 fields>

In [33]:
%%sql
SELECT * FROM customer_delta;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 39, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 3 fields>

In [34]:
%%sql
-- Get the history of the table
DESCRIBE HISTORY customer_delta;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 40, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 15 fields>

In [35]:
%%sql
-- Create a new table from a specific version
CREATE TABLE customer_delta_tmp AS
SELECT * FROM customer_delta VERSION AS OF 2;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 41, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [36]:
%%sql
SELECT * FROM customer_delta_tmp;

StatementMeta(, a9c709dd-b090-4473-806f-30cb89967abe, 42, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 3 fields>

##### **<span style="color:green">Non-Persisted Transactional Tables</span>**

In [ ]:
#cant leverage sql on a dataframe
customers = spark.read.format("csv").option("header","true").load("Files/sparksql/csv/customers.csv")

In [ ]:
# will fail
result = spark.sql("SELECT * FROM customers")
result.show()

**<u>createOrReplaceTempView:</u>**

- **Scope**: The temporary view is scoped to the current Spark session.
- **Lifetime**: The view exists only for the duration of the Spark session in which it was created.
- **Usage**: Suitable for intermediate data processing within a single session.
- **Read-Only**: Cannot directly update or insert rows.

In [ ]:
customers.createOrReplaceTempView("customers_tmp_view")

In [ ]:
result = spark.sql("SELECT * FROM customers_tmp_view")
result.show(3)

<u>**createOrReplaceGlobalTempView:**</u>

- **Scope**: The global temporary view is accessible across all Spark sessions within the same Spark application.
- **Lifetime**: The view persists as long as the Spark application is running.
- **Usage**: Ideal for sharing data across multiple sessions or notebooks within the same application.
- **Read-Only**: Cannot directly update or insert rows.

In [ ]:
customers.createOrReplaceGlobalTempView("customers_glb_tmp")

In [ ]:
result = spark.sql("SELECT * FROM global_temp.customers_glb_tmp")
result.show(3)

Both **createOrReplaceTempView** and **createOrReplaceGlobalTempView** create **read-only** views in the Microsoft Fabric Spark catalog

In [ ]:
%%sql
--customers_tmp_view
UPDATE global_temp.customers_glb_tmp
SET Name='this should fail'
WHERE CustomerID=2;

In [ ]:
%%sql
--this should fail
INSERT INTO global_temp.customers_glb_tmp
VALUES (32,'Jean Joseph','JeanJoseph@sample.com','+1-403-708-9975','Haiti','Product B',55.33,14,774.62,'2025-01-05')

##### **<span style="color:green">Persisted Transactional Tables</span>**

<u>**Delta Tables**</u>

**Purpose**: Provides ACID transactions, scalable metadata handling, and unifies streaming and batch data processing.

**Usage**: Ideal for large-scale data processing and analytics.

**Example**:
```python
df.write.format("delta").saveAsTable("delta_table")

In [ ]:
customers.write.format("delta").mode("overwrite").saveAsTable("customers")

In [ ]:
%%sql
SELECT 
      SPLIT(name,' ')[0] AS FirstName
     ,SPLIT(name,' ')[1] AS LastName
FROM customers;

In [ ]:
%%sql

SELECT
    TransactionDate,
    FIRST_VALUE(SoldPrice) OVER (PARTITION BY TransactionDate ORDER BY TransactionDate) AS FirstClosePrice,
    LAST_VALUE(SoldPrice) OVER (PARTITION BY TransactionDate ORDER BY TransactionDate ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS LastClosePrice
FROM
    customers_delta
ORDER BY TransactionDate;


**SQL performance optimization**

In [ ]:
%%sql

SELECT
    TransactionDate,
    FIRST_VALUE(SoldPrice) OVER (PARTITION BY TransactionDate ORDER BY TransactionDate) AS FirstClosePrice,
    LAST_VALUE(SoldPrice) OVER (PARTITION BY TransactionDate ORDER BY TransactionDate ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS LastClosePrice
FROM
    customer_delta
ORDER BY TransactionDate;


#### Insights into Spark SQL Query Execution and Optimization

Besides the `EXPLAIN` command, there are several other ways to gain insights into Spark SQL query execution and optimization:

##### 1. Spark UI
The Spark UI provides detailed information about the execution of your Spark jobs, including stages, tasks, and the Directed Acyclic Graph (DAG) of operations. You can access the Spark UI by navigating to the URL provided when you start your Spark application.

##### 2. Query Execution Listeners
You can use Spark's `QueryExecutionListener` to capture detailed information about query execution, including logical and physical plans. This allows you to programmatically access execution details.

##### 3. Logging
Enable detailed logging for the Catalyst optimizer to capture information about the optimization process. You can configure Spark's logging settings to include more detailed logs.

##### 4. DataFrame API
The DataFrame API provides methods like `explain()` that can be used to print the logical and physical plans of a DataFrame. You can specify different modes to get various levels of detail:

```python
df.explain(mode="simple")      # Prints only the physical plan
df.explain(mode="extended")    # Prints logical and physical plans
df.explain(mode="codegen")     # Prints physical plan and generated code
df.explain(mode="cost")        # Prints logical plan and statistics
df.explain(mode="formatted")   # Splits explain output into two sections: a physical plan outline and node details


In [ ]:
%%pyspark
# Define the SQL query
sql_query = """
SELECT 
  ROW_NUMBER() OVER (ORDER BY email) AS Id,
  SUBSTRING(email, 1, INSTR(email, '@') - 1) AS username
FROM sales_json
"""

# Execute the query
result = spark.sql(sql_query)

# Display the result
result.explain(mode="simple") 